In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import cv2
import sys
sys.path.append("game/")
import wrapped_flappy_bird as game

import numpy as np
import collections
import random
from collections import deque

In [2]:
conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=2)

In [3]:
empty_frame = np.zeros((80, 80), dtype=np.float32)
empty_state = np.stack((empty_frame, empty_frame, empty_frame, empty_frame), axis=0)

In [4]:
x = np.stack((empty_state,empty_state), axis=0)
print(np.shape(x))

(2, 4, 80, 80)


In [5]:
x = torch.from_numpy(x)

In [6]:
x = torch.autograd.Variable(torch.randn(2, 4, 80, 80))

In [7]:
x = conv1(x)

In [8]:
x.size()

torch.Size([2, 32, 20, 20])

In [9]:
conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)

In [10]:
x = conv2(x)

In [11]:
x.size()

torch.Size([2, 64, 10, 10])

In [12]:
map_size = (64, 10, 10)

In [13]:
fc1 = nn.Linear(map_size[0]*map_size[1]*map_size[2], 256)

In [14]:
x = x.view(x.size()[0],-1)

In [15]:
x.size()

torch.Size([2, 6400])

In [16]:
x = fc1(x)

In [17]:
x.size()

torch.Size([2, 256])

In [18]:
fc2 = nn.Linear(256, 2)

In [19]:
x = fc2(x)

In [20]:
x.size()

torch.Size([2, 2])

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.map_size = (64, 10, 10)
        self.fc1 = nn.Linear(self.map_size[0]*self.map_size[1]*self.map_size[2], 256)
        self.fc2 = nn.Linear(256, 2)

    def foward(self, x):
        #foward procedure to get MSE loss
        x = F.relu(self.conv1(x), inplace=True)
        x = F.relu(self.conv2(x), inplace=True)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x), inplace=True)
        x = self.fc2(x)
        return x

In [22]:
x = torch.autograd.Variable(torch.randn(2, 4, 80, 80))

In [25]:
model = Net()

In [27]:
model

Net (
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (fc1): Linear (6400 -> 256)
  (fc2): Linear (256 -> 2)
)

In [29]:
model.foward(x)

Variable containing:
1.00000e-02 *
  2.2144  4.5368
  5.6918  4.9775
[torch.FloatTensor of size 2x2]

In [30]:
x.size()

torch.Size([2, 4, 80, 80])